In [1]:
import pandas as pd 
import numpy as np 
import scipy
# import xlrd 
import sklearn

from Gibbs_model import Gibbs_sampling

from sklearn.model_selection import train_test_split
from scipy.stats import multivariate_normal
from utils import baseline_lr,baseline_esnet,baseline_lrCV,baseline_esnetCV,baseline_justmean
from sklearn.model_selection import KFold
from scipy.stats import binom 
from scipy.stats import norm
from sklearn.linear_model import Ridge
from tqdm import trange
import time

In [3]:
# data_loading 
np.random.seed(123)

data_table = pd.read_csv('../data/processed/all_feature_p2_lip_specie.csv')
# data_table = pd.read_csv('../data/processed/all_feature_p1_lip_specie.csv')
target = 'gap_death_consent'


# normalization

# min-max
# df = data_table[target]
# data_table[target] = (df-df.min())/(df.max()-df.min())

# log
df = data_table[target]
data_table[target] =np.log(df + 1.0)
# data_table[target] = (data_table[target]-data_table[target].mean())/data_table[target].std()

# check nan
data_table[target].isnull().values.any()

data_table.fillna(data_table.mean(), inplace=True) # fill nan with column mean

In [4]:
# gene_feature = data_table.iloc[:,1:-368]#for p2
gene_feature = data_table.iloc[:,1:775]#for p1

sur_time = data_table[target].values
Y = sur_time


In [5]:
gene_feature

,lip_Sph(d16:1)_27,lip_Sph(d18:1)_27,lip_Sph(d18:2)_27,lip_S1P(d16:1)_6,lip_S1P(d18:0)_6,lip_S1P(d18:1)_6,lip_S1P(d18:2)_6,lip_Cer(d18:0/16:0)_3,lip_Cer(d18:0/18:0)_3,lip_Cer(d18:0/20:0)_3,...,lip_TG(O-54:4) [NL-18:2]_13,lip_Ubiquinone_24,lip_CE(18:2) [+OH]_15,lip_CE(20:4) [+OH]_15,lip_CE(22:6) [+OH]_15,lip_LPC(18:2) [+OH]_15,lip_LPC(20:4) [+OH]_15,lip_LPC(22:6) [+OH]_15,lip_PC(34:2) [+OH]_15,lip_PC(36:4) [+OH]_15
0,1.508517,-1.142981,-0.370896,1.161898,1.393186,2.439157,3.142364,2.807684,1.906719,1.964852,...,0.036007,0.359336,-0.383320,-0.018562,-0.205400,-0.186943,-0.090134,-0.100703,-0.293183,1.595553
1,0.662510,-0.169350,0.276919,-0.420598,0.890299,1.172982,0.270768,1.235381,-0.120123,0.356863,...,-0.656573,-0.557216,4.805999,2.009266,2.187153,11.290922,7.801393,8.273906,10.974755,2.317701
2,0.382097,0.034175,-0.873632,0.263159,0.442293,-0.246913,-0.094627,0.488499,-1.068774,-0.575590,...,2.257551,-0.525753,-0.165085,-0.231418,-0.152552,-0.118845,-0.172070,-0.142195,-0.062192,0.139186
3,-0.505375,-0.856168,-0.309318,-0.876899,0.129396,-0.332582,-0.197593,1.159958,1.085276,0.743728,...,1.085764,0.103507,-0.345465,-0.515169,-0.190832,-0.202784,-0.200526,-0.244971,-0.218103,-1.082805
4,-0.973029,-1.221808,0.084801,1.401033,0.822457,0.869329,0.284968,0.232719,0.030465,-0.985736,...,-1.369647,-1.081999,5.153219,7.136091,8.406937,4.119153,6.956574,8.631769,6.050933,3.055967
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,0.836448,0.078052,-0.680276,-0.222538,-1.176785,-1.582970,-1.824283,0.018785,1.688315,0.036113,...,-0.919553,-0.499796,0.454661,0.274775,-0.177827,-0.122108,-0.138097,-0.105071,-0.128006,-0.260284
140,0.840385,0.370737,-0.795050,-0.544087,-0.086341,-0.931198,-1.267266,-0.992808,0.144297,-0.843535,...,-0.504697,-0.009101,0.544398,-0.174427,-0.209744,-0.171430,-0.183107,-0.169494,-0.137458,-0.804552
141,0.958798,-0.450028,0.631433,1.413756,-0.018409,-0.517109,0.988292,-1.232623,0.418946,-0.962435,...,-0.217015,0.367528,0.289700,0.035265,-0.179299,-0.172936,-0.163742,-0.136950,-0.152169,-0.558493
142,-0.486482,-0.715181,-0.765942,-1.725063,-1.572528,-1.536034,-1.350552,-1.357425,-1.695333,-0.907401,...,-0.470357,-0.925047,0.599696,-0.066377,-0.217330,-0.155953,-0.172993,-0.199861,-0.191259,-0.745833


In [6]:
feature_names = list(gene_feature.columns)
K=41 # group number, from data process notebook
group_ind_dict = {}
group_ind = []
group_ind_concat = []
for i in range(K):
    group_ind_dict[i] = []

for name in feature_names:
    id = int(name.split('_')[-1])
    group_ind_dict[id].append(name)

for i in range(K):
    group_ind.append(group_ind_dict[i])
    group_ind_concat = group_ind_concat + group_ind_dict[i]

# group_ind

In [7]:
# re-arrange the features of X based on the group split order
X_new = gene_feature[group_ind_concat].values

N_sample,_ = X_new.shape
# add all-one column at the last 
bias_col = np.ones(N_sample).reshape((N_sample,1))
X_new = np.concatenate((X_new,bias_col),axis=1)

print(X_new.shape)

(144, 775)


In [14]:
# init hyper-parameters
alpha = 0.7
beta = 0.7
r0 = 1e-3
r1 = 1.0
a0 = 1.0
b0 = 1.0
JITTER = 1e-3

INTERVAL = 100
VALITA_INTERVAL = 200
BURNING = 3000
MAX_NUMBER = 5000

hyper_paras = {'INTERVAL':INTERVAL, 'BURNING':BURNING,'MAX_NUMBER':MAX_NUMBER,'VALITA_INTERVAL':VALITA_INTERVAL,
'alpha':alpha, 'beta':beta,'r0':r0,'r1':r1,'JITTER':JITTER}

In [8]:

N = 50
lr_rmse = np.zeros(N)
esnet_rmse = np.zeros(N)
mean_rmse = np.zeros(N)

for i in range(N):
    X_train, X_test, y_train, y_test = train_test_split(X_new, Y.squeeze(),test_size=0.5)

    y_mean_tr = y_train.mean()
    y_std_tr = y_train.std()
    y_nor_tr = (y_train-y_mean_tr)/y_std_tr
    data_dict = {'X_tr':X_train, 'y_nor_tr':y_nor_tr, 'X_test':X_test, 'y_test':y_test,'y_mean_tr': y_mean_tr, 'y_std_tr':y_std_tr}    
    w_lr,lr_rmse[i] = baseline_lrCV(data_dict)
    w_es,esnet_rmse[i] = baseline_esnetCV(data_dict)
    mean_rmse[i] = baseline_justmean(data_dict)

print('lr_rmse_mean: %.4f,lr_rmse_std: %.4f '%(lr_rmse.mean(),lr_rmse.std() ) )
print('esnet_rmse_mean: %.4f,esnet_rmse_std: %.4f '%(esnet_rmse.mean(),esnet_rmse.std() ) )
print('just-mean_rmse_mean: %.4f,mean_rmse_std: %.4f '%(mean_rmse.mean(),mean_rmse.std() ) )

  positive)
G:\anaconda\envs\pt\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.010116891867582911, tolerance: 0.005951692591439892
  positive)
G:\anaconda\envs\pt\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0072613163792305535, tolerance: 0.005951692591439892
  positive)
G:\anaconda\envs\pt\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.01431715627117125, tolerance: 0.005951692591439892
  positive)
G:\anaconda\envs\pt\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations.

In [73]:
predict =  data_dict['y_mean_tr']
rmse = np.sqrt(np.mean((predict-data_dict['y_test'].squeeze())**2))
print('with just-mean, rmse is %.5f'%(rmse))


with just-mean, rmse is 1.38379


In [9]:
# # init parameters
# z_array_init = np.random.binomial(size=K, n=1, p= alpha)
# s_list_init = [np.random.binomial(size=len(item), n=1, p= beta) for item in group_ind]
# b_init = np.random.normal(loc=0.0, scale=r1,size=None)
# tau_init = np.random.gamma(shape=alpha, scale=1.0/beta, size=None)
# W_init = []

# for i in range(K):
#     mask1 = 1-z_array_init[i] * s_list_init[i]
#     mask2 = z_array_init[i] * s_list_init[i]
#     spike = np.random.normal(loc=0.0, scale=r0,size=len(s_list_init[i]))
#     slab = np.random.normal(loc=0.0, scale=r1,size=len(s_list_init[i]))
#     W_group = spike * mask1 + slab * mask2

#     W_init.append(W_group)

# init_paras = {'z':z_array_init, 's':s_list_init, 'b':b_init, 'tau':tau_init, 'W':W_init,'a0':a0,'b0':b0}

In [33]:
# init parameters with lr_result
def get_init_paras(w_lr):
    z_array_init = np.ones(K) #np.random.binomial(size=K, n=1, p= alpha)
    s_list_init = [np.random.binomial(size=len(item), n=1, p= beta) for item in group_ind]
    b_init = w_lr[-1]#np.random.normal(loc=0.0, scale=r1,size=None)
    tau_init = 1.0#np.random.gamma(shape=alpha, scale=1.0/beta, size=None)
    W_init = []
    offset=0
    for i in range(K):
        # mask1 = 1-z_array_init[i] * s_list_init[i]
        # mask2 = z_array_init[i] * s_list_init[i]
        # spike = np.random.normal(loc=0.0, scale=r0,size=len(s_list_init[i]))
        # slab = np.random.normal(loc=0.0, scale=r1,size=len(s_list_init[i]))
        # W_group = spike * mask1 + slab * mask2

        
        group_len = len(s_list_init[i])
        W_group= w_lr[offset:offset+group_len]
        offset = offset + group_len
        W_init.append(W_group)

    init_paras = {'z':z_array_init, 's':s_list_init, 'b':b_init, 'tau':tau_init, 'W':W_init,'a0':a0,'b0':b0}
    return init_paras

In [34]:
get_init_paras(w_lr)['tau']

1.0

In [12]:
# split train & test
N=5
lr_rmse = np.zeros(N)
esnet_rmse = np.zeros(N)
ours_rmse = np.zeros(N)
for i in range(5):
    X_train, X_test, y_train, y_test = train_test_split(X_new, Y.squeeze(),test_size=0.30)
    data_dict = {'X_tr':X_train, 'y_tr':y_train, 'X_test':X_test, 'y_test':y_test}      
    w_lr,lr_rmse[i] = baseline_lr(data_dict)
    w_es,esnet_rmse[i] = baseline_esnet(data_dict)
    model = Gibbs_sampling(data_dict,get_init_paras(w_lr), hyper_paras)
    model.model_run()
    ours_rmse[i] = model.model_test()

print('lr_rmse_mean: %.4f,lr_rmse_std: %.4f '%(lr_rmse.mean(),lr_rmse.std() ) )
print('esnet_rmse_mean: %.4f,esnet_rmse_std: %.4f '%(esnet_rmse.mean(),esnet_rmse.std() ) )
print('ours_rmse_mean: %.4f,ours_rmse_std: %.4f '%(ours_rmse.mean(),ours_rmse.std() ) )

  0%|          | 1/5000 [00:00<08:58,  9.28it/s]with lr, rmse is 1.23931
with elanet, rmse is 0.90610

 running test-rmse = 1.31665
running train-rmse = 0.72845

  4%|▍         | 202/5000 [00:21<08:22,  9.55it/s]
 running test-rmse = 1.17275
running train-rmse = 0.80951

  8%|▊         | 402/5000 [00:43<09:06,  8.41it/s]
 running test-rmse = 1.22125
running train-rmse = 0.83236

 12%|█▏        | 602/5000 [01:05<07:55,  9.24it/s]
 running test-rmse = 1.64747
running train-rmse = 0.78348

 16%|█▌        | 802/5000 [01:26<08:27,  8.28it/s]
 running test-rmse = 1.38720
running train-rmse = 1.02710

 20%|██        | 1002/5000 [01:50<08:19,  8.00it/s]
 running test-rmse = 1.00437
running train-rmse = 0.95954

 24%|██▍       | 1202/5000 [02:14<06:40,  9.48it/s]
 running test-rmse = 1.16492
running train-rmse = 0.91064

 28%|██▊       | 1402/5000 [02:36<06:50,  8.76it/s]
 running test-rmse = 0.87052
running train-rmse = 0.85004

 32%|███▏      | 1602/5000 [02:58<06:05,  9.30it/s]
 running test

In [21]:
print('lr_rmse_mean: %.4f,lr_rmse_std: %.4f '%(lr_rmse[[0,2,3]].mean(),lr_rmse[[0,2,3]].std() ) )
print('esnet_rmse_mean: %.4f,esnet_rmse_std: %.4f '%(esnet_rmse[[0,2,3]].mean(),esnet_rmse[[0,2,3]].std() ) )
print('ours_rmse_mean: %.4f,ours_rmse_std: %.4f '%(ours_rmse[[0,2,3]].mean(),ours_rmse[[0,2,3]].std() ) )

lr_rmse_mean: 1.3432,lr_rmse_std: 0.1101 
esnet_rmse_mean: 1.1714,esnet_rmse_std: 0.2027 
ours_rmse_mean: 1.1854,ours_rmse_std: 0.1600 


(array([ 0, 24, 26], dtype=int64),)

In [18]:
model.s_mean[0]

array([0, 1, 1, 0, 1, 1])

In [86]:
from sklearn.linear_model import ElasticNet

clf = ElasticNet(alpha = 0.2,l1_ratio =0.1, fit_intercept=False)
clf.fit(data_dict['X_tr'], data_dict['y_tr'])
predict = clf.predict(data_dict['X_test']).squeeze()
rmse = np.sqrt(np.mean((predict-data_dict['y_test'].squeeze())**2))
print('with elanet, rmse is %.5f'%(rmse))
len(clf.coef_[clf.coef_>0])

with elanet, rmse is 0.92433


114

In [14]:
w_es

array([ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        , -0.        , -0.        , -0.        , -0.        ,
       -0.        , -0.        , -0.02290048, -0.00710251, -0.        ,
       -0.        , -0.00906848, -0.        , -0.00073689, -0.        ,
       -0.01301175,  0.        ,  0.        , -0.        ,  0.04461527,
       -0.        ,  0.        , -0.        , -0.        , -0.        ,
       -0.        ,  0.        ,  0.        , -0.        ,  0.04582258,
       -0.        , -0.        ,  0.        , -0.00123495, -0.00794974,
       -0.01668715, -0.        , -0.        ,  0.        ,  0.        ,
       -0.        , -0.        , -0.        , -0.        , -0.        ,
       -0.05419244,  0.        , -0.        , -0.05123613,  0.00319208,
       -0.11459053,  0.05525388,  0.        ,  0.02155437,  0.        ,
        0.02790468,  0.        , -0.        , -0.        ,  0.        ,
       -0.        , -0.        ,  0.        ,  0.        ,  0.  